Chicago JSON analysis
- Let's create the dataframe and see what shape we're working with

In [102]:
import json
import os
import pandas as pd
import random
import time
import sys

In [103]:
def get_jsons(root):
    return [os.path.join(root,x) for x in os.listdir(root) if x.endswith('.json')]

In [104]:
def get_data(path): 
    return json.loads(open(path).read())

In [105]:
def make_df(data):
    df = pd.DataFrame(data)
    return df[df['name_id'].notna()]

In [106]:
def get_names(df):
    # get unique filenames
    fns = df['file'].unique()

    output = []
    for file in fns:
        subset = df[df['file'] == file]
        ids = subset['name_id'].unique()
        names = []
        for i in ids:
            l = list(subset[subset['name_id'] == i]['word'])
            names.append(' '.join(l))
        # remove duplicates
        names = [x.title() for x in list(set(names))]
        for name in names:
            output.append((file,name))
            
    return output

In [107]:
def process_json(json_path):
    start = time.time()
    data = get_data(json_path)
    df = make_df(data)
    names = get_names(df)
    out_path = '/scratch/groups/malgeehe/celebs/chicago_people/full_names'
    filename = os.path.split(json_path)[-1].split('.json')[0] + '_names.tsv'
    out = os.path.join(out_path,filename)
    with open(out,'w') as f:
        for name in names:
            f.write('{}\t{}\n'.format(name[0],name[1]))
        f.close()
    print('finished {} in {} sec'.format(os.path.split(json_path)[-1]), round(time.time()-start))
    return

In [108]:
path = [x for x in sys.argv if '/scratch/groups/malgeehe/celebs/chicago_people/json/group_' in x]
if len(path) == 1:
    path = str(path[0])
    process_json(path)
else:
    print('problem with sys.argv: {}'.format(sys.argv))

problem with sys.argv: ['/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/ipykernel_launcher.py', '-f', '/tmp/jupyter/kernel-30ba8e9f-792e-49cc-9fcd-12dac423ff0a.json']
